# Assignment 1: Customer Data Preprocessing

## William Ashton

## Data Processing

In [1]:
# Import libraries
import csv
import json
from datetime import datetime
import os

In [2]:
acw = [] # Initialise list to store user dictionaries in

#### Task 1: Read in provided ACW Data.

In [3]:
file = open('acw_user_data.csv','r') # Open CSV file
reader = csv.reader(file) # Reader object which iterates over lines
headers = next(reader) # Skip the first rows, ignoring headers

In [4]:
error_rows = [] # Initialise a list to store rows with errors in

#### Task 2: Convert into nested structures.

In [5]:
def int_cast(row): # Function which will typecast to integers with error handling
    try: # Try casting the value
        value = int(row)
    except ValueError: # Catch value error exception if it isn't valid
        print("Not a valid integer.") 
    return value # Return casted value

In [6]:
for idx, row in enumerate(reader): # Iterate through each row of the reader
    
    user={} # Initialise new user dictionary for each row
    
    user['address'] = {} # Initialise address key 
    # Initialise and set the value of street, city, and postcode to their corresponding columns
    user['address']['street'] = str(row[0])
    user['address']['city'] = str(row[1])
    user['address']['postcode'] = str(row[2])
    
    # Age, distance commuted, and employer
    user['age'] = int_cast(row[3])
    try:
        user['commute_distance'] = float(row[4])
    except ValueError:
        print("Not a valid float.")
    user['employer'] = str(row[5])
    
    user['cc'] = {} # Initialise credit card key
    # Start date, end date, cc number, cvv, and iban
    user['cc']['start_date'] = str(row[6])
    user['cc']['end_date']= str(row[7])
    user['cc']['number'] = int(row[8])
    user['cc']['cvv'] = int_cast(row[9])
    user['cc']['iban'] = str(row[12])
    
    # Check if dependants column has a value
    if row[10] == "":
        user['dependants'] = 'NaN' # If not, assign a value of 'NaN' to the dictionary key
        error_rows.append(idx+1) # and append the index to a list (plus 1 due to indexing beginning with 0)
    else:
        user['dependants'] = int_cast(row[10]) # Otherwise, assign the given value to the dictionary key
    
    # First name, last name, marital status, pension, retired, salary, and sex
    user['first_name'] = str(row[11])
    user['last_name'] = str(row[13])
    user['marital_status'] = str(row[14])
    user['pension'] = int_cast(row[15])
        
    # Check which strings contain "true" or "false", and cast to booleans
    if row[16] == "False":
        user['retired'] = bool(False)
    else:
        user['retired'] = bool(True)
    
    # Salary and sex
    user['salary'] = int_cast(row[17])
    user['sex']= str(row[18])
    
    user['vehicle']={} # Initialise vehicle key
    # Make, model, year, and type
    user['vehicle']['make'] = str(row[19])
    user['vehicle']['model'] = str(row[20])
    user['vehicle']['year'] = int_cast(row[21])
    user['vehicle']['type'] = str(row[22])
    
    acw.append(user) # Append the completed user dictionary to the list

In [7]:
file.close() # Close CSV file

#### Task 3: Print list where error corrections took place

In [8]:
print("Errors found in rows:",error_rows) # Print rows which have no value in the dependants column

Errors found in rows: [22, 110, 180, 206, 271, 273, 275, 359, 461, 469, 580, 637, 680, 726, 823, 866, 918, 932, 984]


#### Task 4: Write records to a JSON file

In [9]:
# Write all records to new JSON file
with open('processed.json', 'w') as file:
    json.dump(acw, file)

#### Task 5: Write records to different JSON files based on employment

In [10]:
# Initialise lists to store retired and employed users
retired = []
employed = []

In [11]:
for user in acw:
    if user['retired'] == True: # If they're retired, append the user's record to the retired list
        retired.append(user)
    else: # Otherwise, append the user's record to an employed list
        employed.append(user)

In [12]:
# Write records of users who are retired to a new JSON file
with open('retired.json', 'w') as file:
    json.dump(retired, file)

In [13]:
# Write records of users who are employed to a different JSON file
with open('employed.json', 'w') as file:
    json.dump(employed, file)

#### Task 6: Check credit card validity

In [14]:
# Function which takes a user and checks how long they've had their credit card
def check_validity(user): # Takes one argument - a user dictionary object
    
    # Convert strings into datetime objects
    start_date = datetime.strptime(user['cc']['start_date'], "%m/%y")
    end_date = datetime.strptime(user['cc']['end_date'], "%m/%y")
    
    # Calculate number of years between the two dates
    years = end_date.year - start_date.year - (end_date.month < start_date.month)
    
    # If it's older than 10 years, return invalid
    if years > 10:
        return "Invalid"
    else: # Otherwise, return valid
        return "Valid"

In [15]:
remove_ccard = [] # Initialise new list to store invalid users

In [16]:
for idx, user in enumerate(acw): # For each user in the collection, call function
    #print("User",idx+1,"-",check_validity(user))
    if check_validity(user) == "Invalid": # If the function returns invalid, append the user to the list
        remove_ccard.append(user)

In [17]:
# Write records of those who's cards have expired to a new JSON file
with open('remove_ccard.json', 'w') as file:
    json.dump(remove_ccard, file)

#### Task 7: Calculate additional metrics

In [18]:
# Read in processed JSON file, which contains every user
with open('processed.json', 'r') as file:
    processed = json.load(file)

In [19]:
for user in processed: # For each user in the collection, declare a salary_commute variable
    if user['commute_distance'] <= 1: # If their commute distance is less than or equal to 1,
        user['salary_commute'] = user['salary'] # Set to same value as their salary
    else: # Else, divide their salary by their commute distance, to 2 decimal values
        user['salary_commute'] = float("%.2f" % (user['salary']/user['commute_distance']))

In [20]:
# Create a new list of the same values, but sorted by their salary_commute
commute = sorted(processed, key=lambda k: k['salary_commute'])

In [21]:
# Write sorted list to a new JSON file
with open('commute.json', 'w') as file:
    json.dump(commute, file)

## Data Visualisation

In [ ]:
# Import pandas, seaborn and matplotlib libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read in CSV file to a dataframe
acw_df = pd.read_csv('acw_user_data.csv')

In [ ]:
acw_df.head() # View top of data for overview

#### Task 1: Calculate metrics for salary and age

In [ ]:
# Calculate the mean salary (2 decimal places when dealing with money)
float("%.2f" % acw_df['Yearly Salary (£)'].mean())

In [ ]:
# Calculate median age
acw_df['Age (Years)'].median()

#### Task 2: Perform univariate plots

In [ ]:
# Plot histogram of age values, with a binwidth of 5
age_plt = sns.displot(acw_df, x ="Age (Years)", binwidth=5)

In [ ]:
# Plot histogram of dependants values, dropping invalid entries where necessary
dependants_plt = sns.displot(acw_df["Dependants"].dropna(), discrete=True) # Discrete bins to choose appropriate bin breaks

In [ ]:
# Plot histogram of age values, conditioned on marital status
age_plt_2 = sns.displot(acw_df, x="Age (Years)", hue="Marital Status", multiple="dodge")

#### Perform multivariate plots

In [ ]:
# Plot commute distance vs salary
commute_salary_plt = sns.relplot(data = acw_df, x = "Distance Commuted to Work (miles)", y = "Yearly Salary (£)")

In [ ]:
# Plot the same variables, but disregard users who don't commute
acw_df_2 = acw_df[acw_df["Distance Commuted to Work (miles)"] > 0]
commute_salary_plt_2 = sns.relplot(data = acw_df_2, x = "Distance Commuted to Work (miles)", y = "Yearly Salary (£)")

In [ ]:
# Plot age vs. salary
age_salary_plt = sns.relplot(data = acw_df, x = "Age (Years)", y = "Yearly Salary (£)")

In [ ]:
# Plot age vs salary, conditioned by number of dependants
age_salary_plt_2 = sns.relplot(data = acw_df, x = "Age (Years)", y = "Yearly Salary (£)", hue="Dependants")

#### Task 4: Save the plots as images

In [ ]:
figures = {} # Create dictionary of figures
# Each key is the file path, and each value is the saved plot
figures["./age.png"] = age_plt
figures["./dependants.png"] = dependants_plt
figures["./age_2.png"] = age_plt_2
figures["./commute_salary.png"] = commute_salary_plt
figures["./commute_salary_2.png"] = commute_salary_plt_2
figures["./age_salary.png"] = age_salary_plt
figures["./age_salary_2.png"] = age_salary_plt_2

In [ ]:
# For each figure in the dictionary, save it
for k, v in figures.items():
    v.savefig(k)